In [6]:
from flair.models import SequenceTagger
from flair.data import Sentence
import pandas as pd
import xgboost as xgb
import numpy as np

2022-10-26 09:44:45.077515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
data = pd.read_csv('./train.csv')
data.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [3]:
data[data.is_duplicate == 1]

,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
...,...,...,...,...
496574,496575,"Bridgestone （Huizhou）Synthetic Rubber Co., Ltd.","Bridgestone India Pvt., Ltd.",1
496760,496761,Arlanxeo International Group,Arlanxeo Corp.,1
497015,497016,Brenntag Peru S.A.C.,Brenntag Chile Comercial E Industrial Ltda,1
497083,497084,Dow Chemical International Private Ltd.,Dow Chemical Pacific,1


In [23]:
data.groupby(['name_1']).count()

,pair_id,name_2,is_duplicate
name_1,,,
Alfagomma,1,1,1
"TOTAL OIL INDIA PRIVATE LIMITED, TOTAL",121,121,121
(Camso Loadstar ( Pivt ) Ltd.),11,11,11
"*** ПОЛИМАРКЕТ, ООО",1,1,1
1 4103412224 Tel Ex 1 410341 23,132,132,132
...,...,...,...
Репсол,42,42,42
СОДЕКС АГ,1,1,1
ФИЛИАЛ КОМПАНИИ ЭКСОН НЕФТЕГАЗ ЛИМИТЕД,1,1,1


In [4]:
import spacy
NER = spacy.load("en_core_web_sm")

NER вещь интерсеная, но использовать его в одиночку для текущей задачи - бесполезно. Он выявляет сущности в тексте (организации, географические локации, имена, даты). Можно дополнительно к эмбедингам проверять является ли все текущее название названием организации.

In [5]:
NER(data.iloc[1].name_2).ents, NER(data.iloc[1].name_1).ents

((Technocraft Industries,), (Apcotex Industries Ltd.,))

In [6]:
for elem in data.iloc[1562].name_2.split():
    for word in NER(elem).ents:
        print(word.text, word.label_)

FRANCE GPE
ARNAY ORG
LE GPE
DUC ORG


In [7]:
for word in NER(data.iloc[0].name_1).ents:
    print(word.text,word.label_)

Iko Industries Ltd. ORG


 Можно попробовать разные виды эмбедингов, а поверх бинарный класстфикатор

In [16]:
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

In [17]:
### Initialising embeddings (un-comment to use others) ###
# glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
# flair_backward = FlairEmbeddings('news-backward-fast')
# bert_embedding = BertEmbeddings()
#elmo_embedding = ElmoEmbedding()

In [18]:
sentence = Sentence(data.iloc[0].name_1)
# embed words in sentence #
flair_forward.embed(sentence)
for token in sentence:
    print(token.embedding)
z = token.embedding.size()[0]

tensor([-3.2673e-03, -1.0586e-05, -2.9737e-03,  ..., -3.4040e-09,
         3.2332e-05,  1.4211e-01])
tensor([-6.9752e-03, -9.2015e-05,  9.3430e-03,  ..., -4.5189e-09,
         8.6353e-05,  3.7106e-03])
tensor([ 2.3929e-04, -5.3914e-05,  1.2584e-02,  ..., -2.6205e-08,
         2.9059e-04,  5.7078e-03])
tensor([ 1.1926e-03, -3.1575e-05,  2.6557e-03,  ..., -1.4811e-07,
        -6.5442e-06,  5.8404e-03])


In [10]:
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [11]:
data = data[:3000]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data[['name_1', 'name_2']], data.is_duplicate, test_size=0.33, random_state=42)

In [13]:
X_train.shape, X_test.shape

((2010, 2), (990, 2))

In [14]:
def get_embed(X):
    X_embed = []
    s = torch.zeros(0, z)
    # iterating Sentence (tqdm tracks progress) #
    for i in tqdm(range(data.shape[0])):
        tweet = data.iloc[i].name_1
        # empty tensor for words #
        w = torch.zeros(0, z)   
        sentence = Sentence(tweet)

        flair_forward.embed(sentence)
        for token in sentence:
            # storing word Embeddings of each word in a sentence #
            w = torch.cat((w,token.embedding.view(-1,z)),0)
              # storing sentence Embeddings (mean of embeddings of all words)   #
            s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)
        X_embed.append(s)
    return X_embed

In [16]:
# у меня не хватает мощностей опробовать хоть какие-то эмбединги на этой выборке
# можно на меньшей
X_train_embed = get_embed(X_train)

100%|███████████████████████████████████████| 3000/3000 [06:14<00:00,  8.00it/s]


In [ ]:
X_test_embed = get_embed(X_test)

 92%|███████████████████████████████████▊   | 2753/3000 [05:26<00:59,  4.14it/s]

In [1]:
model = xgb.XGBClassifier()
model.fit(np.array(X_train_embed), y_train)

In [ ]:
y_pred = model.predict(X_test_embed)
predictions = [round(value) for value in y_pred]

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test, y_pred)